In [1]:
%load_ext lab_black

In [1]:
import requests, json, random
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from config import weather_api_key
from config import g_key
from config import mapbox_token
from citipy import citipy

# from sklearn.linear_model import LinearRegression
from scipy import stats
from datetime import date

In [2]:
px.set_mapbox_access_token(mapbox_token)

In [3]:
# Build list of random coordinates

coordinate_list = []
counter = 0
number_of_points = 500

while counter < number_of_points:
    lat = round(random.uniform(-90, 90), 6)
    lon = round(random.uniform(-180, 180), 6)
    counter += 1
    coordinate_list.append((lat, lon))

# Assign nearest city to given set of coordinates
cities = []
city_count = 0

for coords in coordinate_list:
    city = citipy.nearest_city(coords[0], coords[1])
    if city.city_name not in cities:
        cities.append(city.city_name)
        city_count += 1

In [4]:
# Obtain weather data for cities in the list

cities_results = []
count = 0
found_count = 0
not_found_count = 0

print("Beginning data retrieval:\n")
print(50 * "-")

for citi in cities:
    endpoint = "http://api.openweathermap.org/data/2.5/weather"
    payload = {"q": citi, "appid": weather_api_key, "units": "imperial"}

    r = requests.get(url=endpoint, params=payload)

    count += 1

    if r.status_code == 200:
        data = r.json()
        cities_results.append(data)
        print("Processing record: ", count, " | ", citi)
        found_count += 1
    else:
        print("Processing record: ", count, " | ", citi, " *Not Found*")
        not_found_count += 1

Beginning data retrieval:

--------------------------------------------------
Processing record:  1  |  hermanus
Processing record:  2  |  vaini
Processing record:  3  |  kapaa
Processing record:  4  |  butaritari
Processing record:  5  |  atuona
Processing record:  6  |  tuni
Processing record:  7  |  dawlatabad
Processing record:  8  |  belushya guba  *Not Found*
Processing record:  9  |  tuktoyaktuk
Processing record:  10  |  kaitangata
Processing record:  11  |  odweyne
Processing record:  12  |  busselton
Processing record:  13  |  saint-augustin
Processing record:  14  |  avarua
Processing record:  15  |  east london
Processing record:  16  |  narsaq
Processing record:  17  |  rikitea
Processing record:  18  |  deputatskiy
Processing record:  19  |  qaanaaq
Processing record:  20  |  ganzhou
Processing record:  21  |  santa eulalia del rio  *Not Found*
Processing record:  22  |  mataura
Processing record:  23  |  antofagasta
Processing record:  24  |  aasiaat
Processing record:  

In [9]:
r.status_code

200

In [10]:
r.url

'http://api.openweathermap.org/data/2.5/weather?q=mugumu&appid=37a1498de740e2228eee5f21decc9230&units=imperial'

In [11]:
found_count

255

In [12]:
not_found_count

26

In [13]:
cities_results[0]

{'coord': {'lon': 19.23, 'lat': -34.42},
 'weather': [{'id': 500,
   'main': 'Rain',
   'description': 'light rain',
   'icon': '10d'}],
 'base': 'stations',
 'main': {'temp': 55,
  'feels_like': 51.73,
  'temp_min': 55,
  'temp_max': 55,
  'pressure': 1012,
  'humidity': 77},
 'visibility': 10000,
 'wind': {'speed': 5.01, 'deg': 213, 'gust': 15.99},
 'rain': {'1h': 0.18},
 'clouds': {'all': 100},
 'dt': 1601046954,
 'sys': {'type': 3,
  'id': 2006131,
  'country': 'ZA',
  'sunrise': 1601008071,
  'sunset': 1601052086},
 'timezone': 7200,
 'id': 3366880,
 'name': 'Hermanus',
 'cod': 200}

In [14]:
city_data = []

for city in cities_results:
    city_data.append(
        {
            "City": city["name"],
            "Country": city["sys"]["country"],
            "Lat": city["coord"]["lat"],
            "Long": city["coord"]["lon"],
            "Temp_max": city["main"]["temp_max"],
            "Humidity": city["main"]["humidity"],
            "Cloudiness": city["clouds"]["all"],
            "Wind_Speed": city["wind"]["speed"],
        }
    )

city_weather = pd.DataFrame(city_data)
city_weather
# city_table.to_csv("data/city_weather_data.csv", index=False)

,City,Country,Lat,Long,Temp_max,Humidity,Cloudiness,Wind_Speed
0,Hermanus,ZA,-34.42,19.23,55.00,77,100,5.01
1,Vaini,TO,-21.20,-175.20,71.60,88,20,6.93
2,Kapaa,US,22.08,-159.32,78.80,78,40,14.99
3,Butaritari,KI,3.07,172.79,82.09,71,10,15.77
4,Atuona,PF,-9.80,-139.03,77.54,77,22,14.63
...,...,...,...,...,...,...,...,...
250,Samaná,DO,19.21,-69.34,89.60,75,20,9.17
251,Srednekolymsk,RU,67.45,153.68,38.84,78,27,7.00
252,Kamenka,RU,51.32,42.77,63.82,46,0,9.82
253,Guerrero Negro,MX,27.98,-114.06,73.44,70,0,9.24


In [15]:
hotel_df = city_weather.loc[
    (city_weather["Cloudiness"] == 0)
    & (city_weather["Wind_Speed"] < 10)
    & (city_weather["Temp_max"] > 70)
    & (city_weather["Temp_max"] < 80)
]
hotel_df

,City,Country,Lat,Long,Temp_max,Humidity,Cloudiness,Wind_Speed
35,Castro,BR,-24.79,-50.01,79.47,36,0,2.77
246,Fort-Shevchenko,KZ,44.51,50.26,72.70,35,0,6.49
253,Guerrero Negro,MX,27.98,-114.06,73.44,70,0,9.24


In [22]:
hotel_df["Hotel_Name"] = ""
hotel_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Country,Lat,Long,Temp_max,Humidity,Cloudiness,Wind_Speed,Hotel_Name
35,Castro,BR,-24.79,-50.01,79.47,36,0,2.77,
246,Fort-Shevchenko,KZ,44.51,50.26,72.70,35,0,6.49,
253,Guerrero Negro,MX,27.98,-114.06,73.44,70,0,9.24,


In [25]:
hotel_df["Lat"].values[0]

-24.79

In [79]:
r.json()

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [80]:
cities_results[0]

{'coord': {'lon': -97.86, 'lat': 55.74},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 48.2,
  'feels_like': 43.65,
  'temp_min': 48.2,
  'temp_max': 48.2,
  'pressure': 994,
  'humidity': 87},
 'visibility': 10000,
 'wind': {'speed': 5.82, 'deg': 130},
 'clouds': {'all': 90},
 'dt': 1601013540,
 'sys': {'type': 1,
  'id': 935,
  'country': 'CA',
  'sunrise': 1601036619,
  'sunset': 1601079727},
 'timezone': -18000,
 'id': 6165406,
 'name': 'Thompson',
 'cod': 200}

In [26]:
hotel_df["Lat"].values[0]

-24.79

In [27]:
hotel_df["Lat"].values

array([-24.79,  44.51,  27.98])

In [81]:

for city in hotel_df:
    
# r = requests.get("https://maps.googleapis.com/maps/api/geocode/json", params={
#     "address": "University of Texas",
#     "key": "your key"
# })

# data = r.json()

# lat = data["results"][0]["geometry"]["location"]["lat"]

# lng = data["results"][0]["geometry"]["location"]["lng"]

    lat = hotel_df.loc[hotel_df["City"] == cities_results["name"]]
    lng = cities_results.loc[cities_results["name"] == hotel_df["City"]]["coord"]["lon"]
    hotel_df["Lat"].values[0]
    
    lat_lng_str = str(lat) + "," + str(lng)
    
    r = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params={
        "location": lat_lng_str,
        "radius": 5000,
        "type": "restaurant",
        "key": "g_key"
    })
#     hotel_df["Hotel_Name"] = r


AttributeError: 'list' object has no attribute 'loc'

In [ ]:
# geocoordinates
target_coordinates = "43.6187102, -116.2146068"
target_search = "Chinese"
target_radius = 8000
target_type = "restaurant"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [ ]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
url = "http://www.omdbapi.com/"
api_key = "trilogy"

movies = ["Aliens", "Sing", "Moana"]

movies_results = []

for movie_title in movies:
    
    movie_result = requests.get(url, params={
        "apikey": api_key,
        "t": movie_title
    }).json()
    
    print(movie_result["Director"])
    
    movies_results.append(movie_result)